In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ast import literal_eval
import matplotlib.pyplot as plt

In [2]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [5]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

cust_data['article_id'] = 'a'+cust_data['article_id'].astype('str')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
cust_data_2020_9 = cust_data.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index().\
assign(prods = lambda x:  list(x['prods']))


# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 

In [7]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [8]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [9]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [10]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [11]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [12]:
# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
# cust_data_2020_9["prods"].values[:5]

In [13]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [14]:
import gc

del cust_data_2020_9
gc.collect()

del cust_features
gc.collect()

0

In [34]:
del customer_df
gc.collect()


1660

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [16]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
genre = mlb.inverse_transform(genre_mlb[0:1])
genre

[('a176209023',
  'a568601006',
  'a568601043',
  'a607642008',
  'a625548001',
  'a627759010',
  'a656719005',
  'a694736004',
  'a697138006',
  'a745232001',
  'a785186005',
  'a785710001')]

In [ ]:
# model_data['label'] = genre_mlb

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [19]:
model_data.iloc[0:1000,2:]

,age,tran_count,article_count,spend,Accessories,Bags,Cosmetic,Fun,Furniture,Garment Full body,...,Interior textile,Items,Nightwear,Shoes,Socks & Tights,Stationery,Swimwear,Underwear,Underwear/nightwear,Unknown
0,49.0,10,19,0.648983,1.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24.0,7,14,0.704780,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,20.0,3,6,0.166000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,45.0,7,23,0.565203,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,44.0,144,555,25.466898,23.0,0.0,0.0,0.0,0.0,55.0,...,0.0,0.0,5.0,14.0,7.0,0.0,10.0,14.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,22.0,2,6,0.224644,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
996,26.0,1,2,0.016746,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,35.0,13,39,0.976576,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0
998,22.0,8,12,0.382492,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,3.0,2.0,0.0,0.0


In [43]:
X_train = model_data.iloc[0:10000,2:]
y_train = genre_mlb[0:10000]
X_test = model_data.iloc[10000:11000,2:]
y_test = genre_mlb[10000:11000]

In [44]:
model = Sequential()
model.add(Dense(5000, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

In [45]:
X_train.columns

Index(['age', 'tran_count', 'article_count', 'spend', 'Accessories', 'Bags',
       'Cosmetic', 'Fun', 'Furniture', 'Garment Full body',
       'Garment Lower body', 'Garment Upper body', 'Garment and Shoe care',
       'Interior textile', 'Items', 'Nightwear', 'Shoes', 'Socks & Tights',
       'Stationery', 'Swimwear', 'Underwear', 'Underwear/nightwear',
       'Unknown'],
      dtype='object')

In [23]:
# def macro_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost.
#     Average (1 - soft-F1) across all labels.
#     Use probability values instead of binary predictions.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
    
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
#     macro_cost = tf.reduce_mean(cost) # average on all labels
    
#     return macro_cost

In [24]:
# def macro_f1(y, y_hat, thresh=0.7):
#     """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
#     Args:
#         y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
#         thresh: probability value above which we predict positive
        
#     Returns:
#         macro_f1 (scalar Tensor): value of macro F1 for the batch
#     """
#     y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
#     tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
#     fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
#     fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
#     f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     macro_f1 = tf.reduce_mean(f1)
#     return macro_f1

In [35]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss=macro_soft_f1,
#   metrics=[macro_f1], optimizer=sgd)

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"]
)

model.fit(X_train, y_train, epochs=1, batch_size=10)

100/100 [==============================] - 16s 157ms/step - loss: 0.0022 - categorical_accuracy: 0.0030


In [36]:
preds = model.predict(X_test)
preds[preds>=0.7] = 1
preds[preds<0.7] = 0
# score = compare preds and y_test

In [37]:
len(preds[0])

74282

In [38]:
score = model.evaluate(X_test, y_test, batch_size=10)
score

10/10 [==============================] - 1s 35ms/step - loss: 0.0023 - categorical_accuracy: 0.0000e+00


[0.002259773900732398, 0.0]

In [39]:
genre_mlb[0:1].shape

(1, 74282)

In [40]:
preds[0:1].astype(int).shape

(1, 74282)

In [41]:
mlb.inverse_transform(genre_mlb[0:1])

[('a176209023',
  'a568601006',
  'a568601043',
  'a607642008',
  'a625548001',
  'a627759010',
  'a656719005',
  'a694736004',
  'a697138006',
  'a745232001',
  'a785186005',
  'a785710001')]

In [42]:
mlb.inverse_transform(preds[0:1].astype(int))

[()]

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)